In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/function_calling_with_gemma.ipynb)

# Function Calling with FunctionGemma 🤖

This notebook demonstrates how to use the GenAI Processor library to implement a
Function Calling loop with the FunctionGemma model (via Hugging Face
transformers). To ensure safety across all string inputs, both function calling
and response parsing are executed at the token ID level.



## 1. 🛠️ Setup

First, install the GenAI Processors library:

In [ ]:
!pip install genai-processors

As we are going to use FunctionGemma 270m, you have to accept the terms of use
for Gemma. You can accept the license on Hugging Face by clicking on the Agree
and access repository button on the model page at:
http://huggingface.co/google/functiongemma-270m-it.

After you have accepted the license, you need a valid Hugging Face Token to
access the model. If you are running inside a Google Colab, you can securely use
your Hugging Face Token using the Colab secrets otherwise you can set the token
as directly in the login method.

In [ ]:
from google.colab import userdata
from huggingface_hub import login

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN')  # If you are running inside a Google Colab
login(hf_token)

Then, import the required modules and finish setup.

In [ ]:
from genai_processors import content_api
from genai_processors import streams
from genai_processors.core import function_calling
import nest_asyncio

nest_asyncio.apply()  # Needed to run async loops in Colab


def print_part(part: content_api.ProcessorPart):
  if not part.substream_name:
    # default substream - contains what the user would see.
    print(f'{part.text}', flush=True, end='')

  if part.substream_name:
    # subtream_name = "function_call" / internal function calls.
    if part.function_call:
      print(
          f'\033[96m FC: {part.function_call.name}:'
          f' {part.function_call.args}\033[0m ',
          flush=True,
      )
    elif part.function_response:
      print(
          f'\033[96m FR: {part.function_response.response}\033[0m ',
          flush=True,
      )
    else:
      print(f'\033[96m {part}\033[0m ', flush=True, end='')

## 2. 👆 Define the functions to be called

GenAI Processors supports automated function calling, deriving tool definitions
directly from your Python function signatures. Simply define your function with
a descriptive docstring. While introspection captures the basic schema, a
detailed docstring covering arguments and return values is critical for the
model to generate accurate calls.

In [ ]:
def get_temperature_celsius(location: str) -> dict[str, float]:
  """Gets the temperature in Celsius at location including a weather description.

  Args:
    location: name of the city, region or place where the weather is requested.

  Returns:
    The temperature in Celsius.
  """
  return {"temperature": 21}


def to_fahrenheit(temperature_celsius: float) -> float:
  """Gets the temperature in Fahrenheit from a temperature in Celsius.

  Args:
    temperature_celsius: temperature in Celsius.

  Returns:
    The temperature in Fahrenheit.
  """
  return temperature_celsius * 9 / 5 + 32


def get_events(location: str) -> list[str]:
  """Gets the list of events happening today at a given location.

  Args:
    location: name of the city, region or place where the events take place.

  Returns:
    The list of event descriptions.
  """
  return ["City Hall concert at 8pm", "Ice skating show at 10pm"]


# Create the tool list that will be used below for all function calling
# processors.
tool_list = [get_temperature_celsius, to_fahrenheit, get_events]

## 3. ✨ Create the Transformer model

Configure the parameters in the form below:

-   **model_name**: Specify a Hugging Face FunctionGemma model (standard or
    fine-tuned). Any model implementing the FunctionGemma function calling
    format is compatible.

-   **log_chat_template**: Set to True to output the full prompt history to the
    runtime logs. While verbose, this is useful for debugging generated function
    calls and responses.

-   **system_instruction**: use this to guide the model's behavior or refine
    specific function calling requirements.

In [ ]:
from genai_processors.core import transformers_model

model_name = 'google/functiongemma-270m-it'  # @param {type: "string"}
log_chat_template = True  # @param {"type":"boolean"}
system_instruction = 'You are a model that can do function calling with the following functions.'  # @param {type: "string"}

**NOTE**: unless you do fine-tuning on the functions you plan to use, it is
highly recommended to end the system instructions with the sentence "You are a
model that can do function calling with the following functions".

We can now create the transformer model: you only need to add the functions
you'd like to use in the constructor as shown below.

In [ ]:
hf_model = transformers_model.TransformersModel(
    model_name=model_name,
    generate_content_config=transformers_model.GenerateContentConfig(
        tools=tool_list,
        system_instruction=system_instruction,
    ),
    log_chat_template=log_chat_template,
    tool_response_format='dict',
)

## 4. 📦 Wrap the transformer model into a Function Calling loop

Adding automatic function calling to an existing transformer implementing the
FunctionGemma function calling format is done as follows:

In [ ]:
fc = function_calling.FunctionCalling(
    hf_model,
    fns=tool_list,
)

Note that we need to add the functions here. Those functions should be the same
as the one provided during `hf_model` creation. The `FunctionCalling` processor
will handle the function calling loop, the scheduling and execution of functions
as well as the function declarations in the prompt.

## 5. ▶️ Run the function calling processor

The `FunctionCalling` processor is a standard GenAI Processor and can therefore
be used and combined with any other GenAI processor. Here, we typically apply it
to a stream of content, that is simply a piece of text.

The function calls (FC) and function responses (FR) are shown in cyan.

In [ ]:
input_stream = streams.stream_content(['What is the temperature in London?'])

async for part in fc(input_stream):
  print_part(part)

The model is called twice here.

*   First call to produce a Function Call (FC) response that is intercepted by
    the `function_calling` processor. The `get_temperature_celsius()` function
    is executed and function response is returned.
*   Second call to gives an answer to the user based on the function response.

## 6. 🛤️ Parallel calls

FunctionGemma is currently trained to handle a single step and can be used to
call a single function as above or a few functions that can be run in parallel.

In [ ]:
input_stream = streams.stream_content([
    'What is the temperature in London? Give me also the list of events for'
    ' today.'
])

async for part in fc(input_stream):
  print_part(part)

**What's next?** Try it in your own code. Enabling function calling requires
only a few extra lines of setup and well-documented docstrings.